# This jupyter notebook is for assistant maintenance purposes only.

In [8]:
# %pip install --upgrade openai

In [9]:
#import the necessary libraries
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [10]:

client = OpenAI()

# Create the assistant with file_search enabled

Our first step is to create an Assistant that can do file searching regardless of where the vector store resides (Assistant or Thread)

In [11]:
#Creating the Assistant

assistant = client.beta.assistants.create(
    model="gpt-4o-mini",
    
    instructions="You are BisonGPT, a Howard University assistant dedicated to supporting students with questions on academics, university policies, resources, and more. Provide clear, concise, and helpful answers, referencing university guidelines as needed. Focus on academic advising, course schedules, graduation requirements, and campus resources. Maintain a friendly and professional tone to empower students on their academic journeys.",
    
    name="BisonGPT",
    
    tools=[{"type": "file_search"}],
    
    metadata={
        "can_be_used_for_file_search": "True",
        "can_hold_vector_store": "True",
    },
)

# Create a vector store


In [ ]:
vector_store = client.beta.vector_stores.create(name="Howard University Data")

# Add all files or...

In [ ]:
def get_all_files(directory_path):
    file_paths = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

files = get_all_files("../Data/")

file_streams = [open(path, "rb") for path in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

#close the files

for file in file_streams:
    file.close()

completed
FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)


# Or update the vector store with new files

Still testing DO NOT RUN THIS

In [ ]:
# Retrieve and clear out current files

vector_store_files = client.beta.vector_stores.files.list(vector_store_id=vector_store.id)

for file in vector_store_files.data:
    client.beta.vector_stores.files.delete(
        vector_store_id=vector_store.id, file_id=file.id
    )

#Get all the new files

files = get_all_files("../Data/")

file_streams = [open(path, "rb") for path in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Attach the vector store to the assisstant

In [14]:
try:
    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )
except Exception as e:
    print(f"Failed to attach vector store to assistant: {e}")